In [ ]:
import mlrun

In [ ]:
project = mlrun.get_or_create_project(name="schedule-workflow", context="./")

In [ ]:
project.set_source(source="git://github.com/xsqian/mlrun-github-example.git#main", pull_at_runtime=True)

In [ ]:
# %%writefile src/func1.py
# def handler():
#     print("inside func1")
#     return "return from func1"

In [ ]:
fn = project.set_function(
    name="function1",
    func="src/func1.py",
    kind="job",
    image="mlrun/mlrun",
    handler="handler",
    with_repo=True,
)

In [ ]:
fn=project.get_function("function1")

In [ ]:
fn.run(local=True)

In [ ]:
%%writefile src/schedule-workflow.py

from kfp import dsl
import mlrun


# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name="scheduled-pipeline",
    description="Example of scheduled pipeline"
)
def pipeline():
    
    # Get current project
    project = mlrun.get_current_project()

    f1 = project.get_function("function1")

    # Ingest the data set
    f1_rs = project.run_function(
        function=f1
    )

    project.run_function(
        function=f1
    ).after(f1_rs)

In [ ]:
# project.set_workflow(workflow_path='src/schedule-workflow.py',name='testing-schedule',schedule='*/10 * * * *')

In [ ]:
project.save()
project.run('testing-schedule', watch=True, dirty=False, engine="remote")

In [ ]:
project.run('testing-schedule', watch=True, schedule=True)

In [ ]:
mlrun.get_run_db().delete_schedule(project=project.name,name='testing-schedule')